# Testing the `channel` function given in the pdf

In [84]:
import numpy as np
import math

In [87]:
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3) 
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput)), erasedIndex

In [40]:
channelInput = np.array([1,-1,1,1,1,-1])
channelOutput = channel(channelInput)

print(channelOutput)

[ 7.09291894 -2.59928026 -1.48739072  2.76715194 -0.53892032 -2.80916199]


# Command to get the real channel output

In [ ]:
python3 client.py --input_file=in.txt --output_file=out.txt --srv_hostname iscsrv72.epfl.ch --srv_port 80

In [13]:
np.savetxt("in.txt", channelInput)

In [15]:
channelOutput = np.loadtxt("out.txt")
print(channelOutput)

[ 8.86527571  2.56791895 -5.904792   -2.13201648  8.97192555 -0.96314209]


# Functions to get a string to channel input format ([-1,1,1,-1,...]) and back

In [22]:
def strToBits(string):
    res = []
    byte_string = string.encode('utf-8')
    for b in byte_string:
        bit_array = bin(b)[2:]
        bit_array = '00000000'[len(bit_array):] + bit_array
        res.extend(bit_array)
    return res

def stringToChannelInput(string):
    bits = np.array(strToBits(string), dtype='int64')
    return 2*bits - 1

In [40]:
channelOutput = stringToChannelInput("Ô¦·`lÃ9ö¦½@bb¼ç7Øq*z÷:ét'f{-Xf	TMycçí|MÌ88_:l(\°ûÌêÙSòt8ã`}ÄÐI¯")
print(channelOutput)
print(len(channelOutput))

[ 1  1 -1 -1 -1 -1  1  1  1 -1 -1  1 -1  1 -1 -1  1  1 -1 -1 -1 -1  1 -1
  1 -1  1 -1 -1  1  1 -1  1  1 -1 -1 -1 -1  1 -1  1 -1  1  1 -1  1  1  1
 -1  1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1 -1 -1  1  1 -1 -1 -1 -1  1 -1
  1 -1 -1 -1 -1  1 -1 -1  1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1 -1 -1  1  1
 -1 -1  1  1  1 -1 -1  1  1  1 -1 -1 -1 -1  1  1  1 -1  1  1 -1  1  1 -1
  1  1 -1 -1 -1 -1  1 -1  1 -1 -1  1  1  1 -1 -1  1  1 -1 -1 -1 -1  1 -1
  1 -1  1 -1 -1  1  1 -1 -1 -1 -1  1 -1 -1  1 -1  1  1 -1 -1 -1 -1  1 -1
  1 -1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1
 -1  1  1 -1 -1 -1  1 -1  1  1 -1 -1 -1 -1  1 -1  1 -1  1  1  1  1 -1 -1
 -1 -1 -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1  1  1  1
 -1 -1  1  1 -1  1  1  1  1  1 -1 -1 -1 -1  1  1  1 -1 -1  1  1 -1 -1 -1
 -1  1  1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  1  1  1 -1  1 -1
  1  1 -1 -1 -1 -1  1  1  1 -1  1  1 -1  1  1  1 -1 -1  1  1  1 -1  1 -1
  1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1  1  1

In [35]:
def channelOutputToString(channelOutput):
    bits = ((channelOutput+1)/2).astype('int64').tolist()
    byte_string = ""
    for char_index in range(len(bits)//8):
        bit_list = bits[char_index*8:(char_index+1)*8]
        byte = chr(int(''.join([str(bit) for bit in bit_list]), 2))
        byte_string += byte
    return byte_string

    

In [41]:
print(channelOutputToString(channelOutput))

ÃÂ¦Â·`lÂÃ9Ã¶ÂÂ¦Â½@bbÂ¼Ã§7Ãq*zÃ·:Ã©Ât'f{-ÂXf	ÂTMycÃ§Ã­|MÃ88_:l(\Â°Ã»ÃÃªÃÂSÂÃ²t8Ã£`}ÃÃIÂÂ¯


# Encoding

In the homework, they talk about "4 equally likely messages". Hence I propose to encode every pair in the message into a triple. This effecitvely makes the messages 1.5 longer. 

* For messages that are of size $2k$:
    We simply do the pair to triple transformation as stated above. We obtain a signal of size $3k$

* For messages that are of size $2k-1$:
    We do the same transform as above for the first $2(k-1)$. Then we double the last character. We therefore obtain a message of size $3(k-1)+2=3k-1$. On the decoder, we can infer $J$ and still recover the last character no matter what. We also know that the input message in of odd size as the decoder will receive a message of size not divisible by $3$.

In [6]:
codewords = [[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1]]
tupleToCodewordDict = {(1, 1): codewords[0], (1, -1): codewords[1], (-1, 1): codewords[2], (-1, -1): codewords[3]}

# The channel only accepts -1 and 1
def transformTuple(subarray):
    return tupleToCodewordDict[tuple(subarray)]

def augmentSignal(array, K):
    n = array.shape[0]
    newArray = np.array([], dtype='int64')
    for i in range(n//2):
        newArray = np.append(newArray, transformTuple(array[2*i:2*i+2]) * K)

    if n % 2 != 0:
        newArray = np.append(newArray, [array[n-1], array[n-1]] * K)

    return newArray

# array is an np.array with values -1 and 1.
# The output is an np.array with values -1 and 1 (channel only accepts those)
def encode(array, K=10):
    return augmentSignal(array, K)

We will employ the straightforward decoding specified in the last subquestion of the homework. In particular, we first guess $J$ and then $H$.

The homework only considered the case where we had one triple.
If the encoded signal is of odd size, we ignore the last two characters. Hence we work with a number of samples divisible by $3$ always (say $3k$). We have:
$$J_{MAP}((y_1, \dotsc, y_k)) = \operatorname{argmin}_{j=1}^{3} \prod_{l=1}^{k} (y_l)_j^2 = \operatorname{argmin}_{j=1}^{3} \sum_{l=1}^{k} \log (y_l)_j$$
In particular, we have a way less error prone $J$ estimator than with a single triple.

In [81]:
def jmap(array):
    p1 = p2 = p3 = 0
    # by '//3' we ignore the last 2 samples if they're here.
    for i in range(len(array)//3):
        p1 += abs(array[3*i])**2
        p2 += abs(array[3*i+1])**2
        p3 += abs(array[3*i+2])**2
    if p1 < p2 and p1 < p3:
        return 0, [p1, p2, p3]
    if p2 < p1 and p2 < p3:
        return 1, [p1, p2, p3]
    return 2, [p1, p2, p3]

def cutJ(array, j, K):
    n = array.shape[0]
    remaining = (n%3)*K
    cutoff = n - remaining

    beforeCutoffIndex = [i%3!=j for i in range(cutoff)]
    afterCutoffIndex = [(i+cutoff)%3!=j for i in range(remaining)]

    left = array[:cutoff]
    right = array[cutoff:]

    return left[beforeCutoffIndex], right[afterCutoffIndex]

def adjacentIndex(j):
    return [[1,2], [0,2], [0,1]][j]

def transformKPairs(subarray, j, K, debug):
    l, r = adjacentIndex(j)
    
    if debug:
        print("Subarray:")
        print(subarray)

    minIndex = -1
    minSum = sys.maxsize
    for i in range(4):
        s0 = 0
        for j in range(K):
            s0 += (subarray[2*j] - codewords[i][l])**2 + (subarray[2*j+1] - codewords[i][r])**2
        if s0 < minSum:
            minSum = s0
            minIndex = i

    if debug:
        print("Min index:")
        print(minIndex)

    if debug:
        print("Result:")
        print(codewords[minIndex])

    return codewords[minIndex][:2]

def reduceSignal(leftReducedArray, rightReducedArray, j, K, verboseDebug):
    n = leftReducedArray.shape[0]
    newArray = np.array([], dtype='int64')

    for i in range(n//(2*K)):
        newArray = np.append(newArray, transformKPairs(leftReducedArray[2*K*i:2*K*(i+1)], j, K, verboseDebug))

    if rightReducedArray.shape[0] != 0:
        codewords_simple = [-1, 1]

        minIndex = -1
        minSum = sys.maxsize
        for i in range(2):
            s0 = 0
            for e in rightReducedArray:
                s0 += (e - codewords_simple[i])**2
            if s0 < minSum:
                minSum = s0
                minIndex = i
                
        if verboseDebug:
            print("Min index:")
            print(minIndex)
            print("Codeword:")
            print(codewords_simple[minIndex])

        newArray = np.append(newArray, codewords_simple[minIndex])

    return newArray 

# array is an np.array with values -1 and 1.
def decode(array, K=10, debug=False, verbose=False):
    n = array.shape[0]
    j, stats = jmap(array)
    if debug:
        print(f"j: {j}")
    leftReducedArray, rightReducedArray = cutJ(array, j, K)
    if debug:
        print(f"after cutJ: {leftReducedArray}")
        print(f"after cutJ: {rightReducedArray}")

    reducedArray = reduceSignal(leftReducedArray, rightReducedArray, j, K, debug and verbose)

    return reducedArray, j, stats

In [25]:
# K has to be coprime with 3.
K = 5

x = np.array([1, 1])
# x = np.array([-1, -1, 1, -1,  1, -1,  1,  1, -1,  1, -1, -1, -1,  1,  1,  1, -1])
# x = np.array([1,1,1,-1,-1,1,-1,-1,1,1])
print("Original:")
print(x)
x = encode(x, K)
print("Encoded:")
print(x)
x, erasedIndex = channel(x)
print("After channel:")
print(x)
x, j, stats = decode(x, K, debug=True)
print("After decoding:")
print(x, j, stats)

Original:
[1 1]
Encoded:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
After channel:
[ 0.99809013 -0.6487678   0.42176822  0.45735516  0.51881293  0.9020472
  0.47732168 -1.07861842  1.19359552  2.21928996  0.89353935  0.62769464
  0.86626238  0.47628124  2.21399959]
j: 1
after cutJ: [0.99809013 0.42176822 0.45735516 0.9020472  0.47732168 1.19359552
 2.21928996 0.62769464 0.86626238 2.21399959]
after cutJ: []
After decoding:
[1 1] 1 [7.108852075018044, 2.8787405879615084, 7.712042606085705]


## Testing the accuracy of the above encoding

In [28]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def stringToChannelInput(string):
    bits = tobits(string)
    bits = [bits[i] for i in range(len(bits)) if i%8!=0]
    bits = np.array(bits, dtype='int64')    
    
    return 2*bits - 1

def channelOutputToString(channelOutput):
    bits = ((channelOutput+1)/2).astype('int64').tolist()

    n = 7
    i = 0
    while i < len(bits):
        bits.insert(i, 0)
        i += (n+1)

    return frombits(bits)

In [4]:
def getRandom1ByteUtf8():
    return [0] + [random.randint(0,1) for i in range(7)]

def getRandomString(length=80):
    packs = [random.randint(0,1) for i in range(length)]
    bits = []
    for i in range(length):
        bits += getRandom1ByteUtf8()

    return frombits(bits)

def strDistance(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            count += 1
    return count


In [88]:
import random

avg_dist = 0
trials = 10
length = 80
K = 71

for i in range(trials):
    inputString = getRandomString(length)
    x = stringToChannelInput(inputString)

    y = encode(x, K)
    print(f"n: {y.shape[0]}")
    y, j = channel(y)
    y, jj, stats = decode(y, K, debug=False)

    delta = y-x

    # if np.any(delta):
        # diff = np.linalg.norm(delta/2, ord=1)
        # print(f"ERROR: n: {n}, diff: {diff}, j: {j}, infered: {jj}, \n (y-x)/2: {(delta/2)}, \n x: {x}, \n y: {y}")
        # avg_diff += diff 
    # else: 
        # print(f"CORRECT: {np.linalg.norm(delta/2, ord=1)/n}")

    outputString = channelOutputToString(y)

    # print(f"encoded size: {y.shape[0]}, input: {inputString}, output: {outputString}")

    dist = strDistance(inputString, outputString)
    print(f"dist: {dist}")
    avg_dist += dist

    if j != jj:
        errs+=1
        print("ERROR IN J ESTIMATION")
        # print(f"j: {j}, infered: {jj}")
        # print(f"stats: {stats}")
    # elif not np.array_equal(x, y):
    #     errs+=1
    #     print(f"ERROR: x: {x}, y: {y}")

print(f"Average distance: {avg_dist/trials}")

n: 59640
dist: 3
n: 59640
dist: 2
n: 59640
dist: 1
n: 59640
dist: 2
n: 59640
dist: 1
n: 59640
dist: 2
n: 59640
dist: 1
n: 59640
dist: 7
n: 59640
dist: 3
n: 59640
dist: 6
Average distance: 2.8


# H-inference, Jonas' style

In [ ]:
decoding_h_choices = [[0,2,3,1], [0,1,3,2], [0,1,2,3]]
def decoder_h(a, b, j):
    if a > 0 and b > 0:
        return decoding_h_choices[j][0]
    elif a > 0 and b < 0:
        return decoding_h_choices[j][1]
    elif a < 0 and b > 0:
        return decoding_h_choices[j][2]
    elif a < 0 and b < 0:
        return decoding_h_choices[j][3]

def transformKPairs(subarray, j, K, debug):
    l, r = adjacentIndex(j)
    
    if debug:
        print("Subarray:")
        print(subarray)

    a = b = 0
    for k in range(K):
        a += subarray[2*k]
        b += subarray[2*k+1]

    infered = codewords[decoder_h(a, b, j)][:2]